## Physical Energy Meter Data Collection

In [1]:
import plotly.express as px
import pandas as pd
import tinytuya
import datetime
import yaml

### 01. Device info
In order to connect with the device, we need to specify a few information like the Device ID, IP and KEY (provided by the Tuya Developer plataform).

In [23]:
# Reading the YAML file with the device info
with open('../secrets.yaml', 'r') as file:
    secrets_info = yaml.safe_load(file)

In [3]:
DEVICE_KEY = secrets_info["DEVICE_KEY"]
DEVICE_ID = secrets_info["DEVICE_ID"]
DEVICE_IP = secrets_info["DEVICE_IP"]
LOCAL_KEY = secrets_info["LOCAL_KEY"]
VERSION = secrets_info["VERSION"]

### 02. Instanciating the device connection

Now we can instanciate a connection to the device.

In [4]:
# Following the lib docs, instanciating an OutletDevice
device = tinytuya.OutletDevice(
    dev_id=DEVICE_ID,
    address=DEVICE_IP,      
    local_key=LOCAL_KEY, 
    version=VERSION
)

device.set_socketPersistent(True)

### 03. Collecting the device data

To collect the energy data, we just have to create loop making multiple requests and saving the results in a dataframe. 

#### 03.1. Creating the output dataframe

In [5]:
# Request responses dataframe
log = pd.DataFrame(columns=['timestamp', 'current', 'power', 'voltage', 'temperature', 'is_on'])

#### 03.2. Formating the response output

In [6]:
def format_device_response(raw_response: any) -> dict:
    """
    Format the JSON device response to a friendly format for the log dataframe.

    Parameters
    ----------
    response: dict
        The raw response fromd device.

    Returns
    -------
    dict:
        Formmatted response.
    """
    data = dict(raw_response)
    timestamp = datetime.datetime.now()

    formatted_data = {
            'temperature': data['dps']['101'] if '101' in data['dps'] else -1, 
            'current': data['dps']['18'] if '18' in data['dps'] else -1, 
            'voltage': data['dps']['20'] if '20' in data['dps'] else -1, 
            'power': data['dps']['19'] if '19' in data['dps'] else -1 ,
            'is_on': data['dps']['1'] if '1' in data['dps'] else -1, 
            'device_id': data['devId'],
            'timestamp': timestamp, 
        }
    
    return formatted_data

#### 03.3. Requests loop

In [7]:
# We can make an infinite loop or define some condition
while(len(log) <= 300):

    # Query the device mesuared data
    payload = device.generate_payload(tinytuya.DP_QUERY)
    device.send(payload)

    # Receive new data. Can be "None"
    data = device.receive()

    # See if any new data is available
    if data:
        formatted_data = format_device_response(data)
        log.loc[len(log)] = formatted_data
        print('Received data -> %r' % data)

    # Request to refresh the data points 
    payload = device.generate_payload(tinytuya.UPDATEDPS, ['18', '19', '20', '101', '1'])
    device.send(payload)
    
    # Receive new data. Can be "None"
    data = device.receive()
    
    # See if any new data is available
    if data:
        data = dict(data)
        ct = datetime.datetime.now()
        formatted_data = format_device_response(data)
        print('Received data -> %r' % data)
        log.loc[len(log)] = formatted_data

Received data -> {'devId': '16315000c45bbe7da87e', 'dps': {'1': True, '9': 0, '18': 409, '19': 464, '20': 1181, '101': 30}}
Received data -> {'devId': '16315000c45bbe7da87e', 'dps': {'18': 380}, 't': 1701665333}
Received data -> {'devId': '16315000c45bbe7da87e', 'dps': {'1': True, '9': 0, '18': 380, '19': 404, '20': 1181, '101': 30}}
Received data -> {'devId': '16315000c45bbe7da87e', 'dps': {'19': 404}, 't': 1701665333}
Received data -> {'devId': '16315000c45bbe7da87e', 'dps': {'1': True, '9': 0, '18': 380, '19': 404, '20': 1181, '101': 30}}
Received data -> {'devId': '16315000c45bbe7da87e', 'dps': {'1': True, '9': 0, '18': 380, '19': 404, '20': 1181, '101': 30}}
Received data -> {'devId': '16315000c45bbe7da87e', 'dps': {'18': 385}, 't': 1701665338}
Received data -> {'devId': '16315000c45bbe7da87e', 'dps': {'19': 413}, 't': 1701665338}
Received data -> {'devId': '16315000c45bbe7da87e', 'dps': {'1': True, '9': 0, '18': 385, '19': 413, '20': 1181, '101': 30}}
Received data -> {'devId': '

#### 03.4. Clean and save the data

In [18]:
# Cleaning the missing data
log = log[(log.power != -1) & (log.current != -1) & (log.voltage != -1)]

# Save the collected data
LOG_FILENAME = '../collected_data/2023_12_04_computer_consumption.csv'
log.to_csv(LOG_FILENAME, index=False)

### 04. Analysing the results

With the dataframe collected we can visualize the results in tables and charts.

In [19]:
log = pd.read_csv(LOG_FILENAME)
log.head()

,timestamp,current,power,voltage,temperature,is_on
0,2023-12-04 01:46:46.721023,409,464,1181,30,1
1,2023-12-04 01:46:46.996234,380,404,1181,30,1
2,2023-12-04 01:46:47.052390,380,404,1181,30,1
3,2023-12-04 01:46:52.116230,380,404,1181,30,1
4,2023-12-04 01:46:52.368182,385,413,1181,30,1


#### 04.1. Plotting the time evolution of the power value

In [22]:
fig = px.line(
    log, 
    y="power", 
    x="timestamp", 
    template='plotly_dark', 
    title='Evolução da Potência em W (somente computador conectado)', 
    labels={'power': 'Potência (W)', 'timestamp': 'Timestamp (HH:MM:SS)'}
)

fig.show()